In [1]:
print('hello')

hello


In [2]:
# CS 143 - SPRING 2021 -- MINIPROJECT
# Skeleton Code

# AT THE BEGINNING OF EACH SESSION, COPY AND PASTE ALL OF THE FOLLOWING CODE
# UNTIL YOU REACH "STOP COPY/PASTE"

from pyspark.sql import SparkSession

import string

import pyspark.sql.functions as F

from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import broadcast, col, date_format, desc, explode, from_unixtime, log, to_date, udf

#         master("spark://spark-master:7077").\
spark = SparkSession.\
        builder.\
        appName("solution").\
        master("local").\
        config("spark.executor.memory", "2048m").\
        getOrCreate()
sc = spark.sparkContext

_PUNCTUATION = "".join(string.punctuation + "’")

# You don't need to do anything with the stopwords, so don't be intimidated.
stopwords = frozenset(["'ll", "'ve", "I", "a", "a's", "able", "about", "above",
    "abst", "accordance", "according", "accordingly", "across", "act",
    "actually", "added", "adj", "affected", "affecting", "affects", "after",
    "afterwards", "again", "against", "ah", "ain't", "all", "allow", "allows", 
    "almost", "alone", "along", "already", "also", "although", "always", "am",
    "among", "amongst", "an", "and", "announce", "another", "any", "anybody",
    "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere",
    "apart", "apparently", "appear", "appreciate", "appropriate",
    "approximately", "are", "aren", "aren't", "arent", "arise", "around", "as",
    "aside", "ask", "asking", "associated", "at", "auth", "available", "away",
    "awfully", "b", "back", "be", "became", "because", "become", "becomes",
    "becoming", "been", "before", "beforehand", "begin", "beginning",
    "beginnings", "begins", "behind", "being", "believe", "below", "beside",
    "besides", "best", "better", "between", "beyond", "biol", "both", "brief",
    "briefly", "but", "by", "c", "c'mon", "c's", "ca", "came", "can", "can't",
    "cannot", "cant", "cause", "causes", "certain", "certainly", "changes",
    "clearly", "co", "com", "come", "comes", "concerning", "consequently",
    "consider", "considering", "contain", "containing", "contains",
    "corresponding", "could", "couldn't", "couldnt", "course", "currently", "d",
    "date", "definitely", "described", "despite", "did", "didn't", "different",
    "do", "does", "doesn't", "doing", "don't", "done", "down", "downwards",
    "due", "during", "e", "each", "ed", "edu", "effect", "eg", "eight", "eighty",
    "either", "else", "elsewhere", "end", "ending", "enough", "entirely",
    "especially", "et", "et-al", "etc", "even", "ever", "every", "everybody",
    "everyone", "everything", "everywhere", "ex", "exactly", "example",
    "except", "f", "far", "few", "ff", "fifth", "first", "five", "fix",
    "followed", "following", "follows", "for", "former", "formerly", "forth",
    "found", "four", "from", "further", "furthermore", "g", "gave", "get",
    "gets", "getting", "give", "given", "gives", "giving", "go", "goes",
    "going", "gone", "got", "gotten", "greetings", "h", "had", "hadn't",
    "happens", "hardly", "has", "hasn't", "have", "haven't", "having", "he",
    "he'd", "he'll", "he's", "hed", "hello", "help", "hence", "her", "here",
    "here's", "hereafter", "hereby", "herein", "heres", "hereupon", "hers",
    "herself", "hes", "hi", "hid", "him", "himself", "his", "hither", "home",
    "hopefully", "how", "how's", "howbeit", "however", "hundred", "i", "i'd",
    "i'll", "i'm", "i've", "id", "ie", "if", "ignored", "im", "immediate", 
    "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed",
    "index", "indicate", "indicated", "indicates", "information", "inner", 
    "insofar", "instead", "into", "invention", "inward", "is", "isn't", "it",
    "it'd", "it'll", "it's", "itd", "its", "itself", "j", "just", "k", "keep",
    "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely",
    "last", "lately", "later", "latter", "latterly", "least", "less", "lest",
    "let", "let's", "lets", "like", "liked", "likely", "line", "little", "look",
    "looking", "looks", "ltd", "m", "made", "mainly", "make", "makes", "many",
    "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely",
    "mg", "might", "million", "miss", "ml", "more", "moreover", "most", "mostly",
    "mr", "mrs", "much", "mug", "must", "mustn't", "my", "myself", "n", "na",
    "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary",
    "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine",
    "ninety", "no", "nobody", "non", "none", "nonetheless", "noone", "nor",
    "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere",
    "o", "obtain", "obtained", "obviously", "of", "off", "often", "oh", "ok",
    "okay", "old", "omitted", "on", "once", "one", "ones", "only", "onto", "or",
    "ord", "other", "others", "otherwise", "ought", "our", "ours", "ourselves",
    "out", "outside", "over", "overall", "owing", "own", "p", "page", "pages",
    "part", "particular", "particularly", "past", "per", "perhaps", "placed",
    "please", "plus", "poorly", "possible", "possibly", "potentially", "pp",
    "predominantly", "present", "presumably", "previously", "primarily",
    "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly",
    "quite", "qv", "r", "ran", "rather", "rd", "re", "readily", "really", 
    "reasonably", "recent", "recently", "ref", "refs", "regarding",
    "regardless", "regards", "related", "relatively", "research",
    "respectively", "resulted", "resulting", "results", "right", "run", "s",
    "said", "same", "saw", "say", "saying", "says", "sec", "second",
    "secondly", "section", "see", "seeing", "seem", "seemed", "seeming",
    "seems", "seen", "self", "selves", "sensible", "sent", "serious",
    "seriously", "seven", "several", "shall", "shan't", "she", "she'd",
    "she'll", "she's", "shed", "shes", "should", "shouldn't", "show", "showed",
    "shown", "showns", "shows", "significant", "significantly", "similar",
    "similarly", "since", "six", "slightly", "so", "some", "somebody",
    "somehow", "someone", "somethan", "something", "sometime", "sometimes",
    "somewhat", "somewhere", "soon", "sorry", "specifically", "specified",
    "specify", "specifying", "still", "stop", "strongly", "sub",
    "substantially", "successfully", "such", "sufficiently", "suggest", "sup",
    "sure", "t", "t's", "take", "taken", "taking", "tell", "tends", "th",
    "than", "thank", "thanks", "thanx", "that", "that'll", "that's", "that've",
    "thats", "the", "their", "theirs", "them", "themselves", "then", "thence",
    "there", "there'll", "there's", "there've", "thereafter", "thereby",
    "thered", "therefore", "therein", "thereof", "therere", "theres",
    "thereto", "thereupon", "these", "they", "they'd", "they'll", "they're",
    "they've", "theyd", "theyre", "think", "third", "this", "thorough",
    "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three",
    "throug", "through", "throughout", "thru", "thus", "til", "tip", "to",
    "together", "too", "took", "toward", "towards", "tried", "tries", "truly",
    "try", "trying", "ts", "twice", "two", "u", "un", "under", "unfortunately",
    "unless", "unlike", "unlikely", "until", "unto", "up", "upon", "ups", "us",
    "use", "used", "useful", "usefully", "usefulness", "uses", "using",
    "usually", "v", "value", "various", "very", "via", "viz", "vol", "vols",
    "vs", "w", "want", "wants", "was", "wasn't", "wasnt", "way", "we", "we'd",
    "we'll", "we're", "we've", "wed", "welcome", "well", "went", "were",
    "weren't", "werent", "what", "what'll", "what's", "whatever", "whats",
    "when", "when's", "whence", "whenever", "where", "where's", "whereafter",
    "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever",
    "whether", "which", "while", "whim", "whither", "who", "who'll", "who's",
    "whod", "whoever", "whole", "whom", "whomever", "whos", "whose", "why",
    "why's", "widely", "will", "willing", "wish", "with", "within", "without",
    "won't", "wonder", "wont", "words", "world", "would", "wouldn't",
    "wouldnt", "www", "x", "y", "yes", "yet", "you", "you'd", "you'll",
    "you're", "you've", "youd", "your", "youre", "yours", "yourself",
    "yourselves", "z", "zero", "ucla", "class", "people", "classes", "lol",
    "thanks", "dm"])

@udf(ArrayType(StringType()))
def cleantext(x):
    # Convert to lower case.
    text = x.lower()
    # Split on space
    tokens = text.split(' ')
    # Remove stopwords
    nostops = ' '.join([token for token in tokens if token not in _STOPWORDS.value])
    # Remove punctuation.
    tokens = ''.join([c for c in nostops if c not in _PUNCTUATION]).split(' ')
    # Remove empty tokens.
    tokens = [token for token in tokens if token]
    return tokens


# UDF is automatically sent to all workers, but stopwords frozenset is not.
# We have to distribute it to the workers ourself:
_STOPWORDS = sc.broadcast(stopwords)

In [3]:
##################################
# Tips for Writing and Debugging #
##################################
# 1. If you want to check that you are on the right track, use the
#    head(10) method. For example:
#    ucla.select("subreddit").show()
#    or ucla.select("subreddit").head(10)
#    will output the subreddit field to the screen.
#    Remember what we discussed about LAZY EVALUATION.
# 2. You can print the schema at any time by adding .printSchema()
# 3. Words in asterisks ** give a clue as to what operator to use. (use documentation)
#    You can probably use other operators as well if you find a better one.
# 4. The imports come from my solution, and this should also give a clue. This does NOT
#    mean you can only use my operators. You may input your own.
# 5. I've prefixed each step with a variable containing your output. You should only need
#    to write one line for each. You are free to use more, but note that you may also be doing
#    something wrong, it just depends on your method.
# 6. Be careful with aliases.
# 8. The usual aggregation functions are implemented: count, countDistinc, min, max, avg etc. These
#    are imported as "F".
# 9. TF-IDF is pretty naive for this problem. Don't expect to be impressed with the results.
##################################

###########################################
# YOU WILL WRITE CODE FOR STEPS 3, 5-14
###########################################

# STEP 1/2: Load the data and subset to only the three columns that we will
# use in this prokect. "body", the text of the comment, "created_utc" which
# is the  Unix timestamp when it was submitted, and "id" which is a unique
# identifier for each comment. We will also drop duplicates because the
# researcher fetched some comments multipe times.
ucla = spark.read.json("data/*.json")  # creates a DataFrame
ucla = ucla.select(["id", "created_utc", "body"]).dropDuplicates()

In [4]:
ucla.show(5)

+-------+-----------+--------------------+
|     id|created_utc|                body|
+-------+-----------+--------------------+
|ezsdavt| 1568087811|If anyone is look...|
|ezxy2ms| 1568252600|            Jack off|
|f00o5zr| 1568304235|    Where’s the joke|
|f026ytg| 1568324562|that’s the minor ...|
|f02oiym| 1568334930|Oh yea that perso...|
+-------+-----------+--------------------+
only showing top 5 rows



In [5]:
# **TODO** STEP 3A (originally Step 6): Remove comments whose entire text is 
# "[deleted]", "deleted", "[removed]" and "removed" which just 
# means that the comment was deleted or removed and is not interesting to us.
# What operator would we use to find these?
result3a = ucla.where("body not in ('[deleted]', 'deleted', '[removed]', 'removed')")
result3a.show(5)

+-------+-----------+--------------------+
|     id|created_utc|                body|
+-------+-----------+--------------------+
|ezsdavt| 1568087811|If anyone is look...|
|ezxy2ms| 1568252600|            Jack off|
|f00o5zr| 1568304235|    Where’s the joke|
|f026ytg| 1568324562|that’s the minor ...|
|f02oiym| 1568334930|Oh yea that perso...|
+-------+-----------+--------------------+
only showing top 5 rows



In [6]:
# **TODO** STEP 3B: Use the UDF to clean the text of each comment by converting
# to lower case and splitting on a space after removing stopwords.  Note that
# splitting on a space gives us a LIST of tokens into the column. Note that
# cleantext is NOT perfect, you will see some bugs.
# HINT: Research withColumn operator, which creates a new column.
# NOTE: You may need to filter out empty rows first.

result3b = result3a.withColumn('cleaned_text',cleantext('body'))
result3b.show(5)

+-------+-----------+--------------------+--------------------+
|     id|created_utc|                body|        cleaned_text|
+-------+-----------+--------------------+--------------------+
|ezsdavt| 1568087811|If anyone is look...|     [tickets, 👀👀]|
|ezxy2ms| 1568252600|            Jack off|              [jack]|
|f00o5zr| 1568304235|    Where’s the joke|      [wheres, joke]|
|f026ytg| 1568324562|that’s the minor ...|[thats, minor, th...|
|f02oiym| 1568334930|Oh yea that perso...|[yea, person, lgb...|
+-------+-----------+--------------------+--------------------+
only showing top 5 rows



In [7]:
# STEP 4: There is a Unix timestamp (a long) in the field "created_utc". Let's
# create a new column "month" that converts the timestamp to a Python datetime object,
# and also only retains the MONTH and YEAR as "2019-11".
# Asking you to write code to convert dates/times would be cruel and unusual punishment,
# so I have done it for you.
result4 = result3b.withColumn("month", date_format(to_date(from_unixtime("created_utc"),
            "yyyy-MM-dd HH:mm:SS"), "yyyy-MM")).drop(result3b.created_utc)
result4.show(5)

+-------+--------------------+--------------------+-------+
|     id|                body|        cleaned_text|  month|
+-------+--------------------+--------------------+-------+
|ezsdavt|If anyone is look...|     [tickets, 👀👀]|2019-09|
|ezxy2ms|            Jack off|              [jack]|2019-09|
|f00o5zr|    Where’s the joke|      [wheres, joke]|2019-09|
|f026ytg|that’s the minor ...|[thats, minor, th...|2019-09|
|f02oiym|Oh yea that perso...|[yea, person, lgb...|2019-09|
+-------+--------------------+--------------------+-------+
only showing top 5 rows



In [8]:
# **TODO** STEP 5: Recall that each row of the column cleaned_text contains a list of
# tokens. We can't do anything with this. *Explode* this list so that each word appears
# as a new row. Then *drop* the cleaned_text column since we don't need it anymore.
result5 = result4.withColumn('word', explode('cleaned_text')).drop('cleaned_text')
result5.show(5)

+-------+--------------------+-------+-------+
|     id|                body|  month|   word|
+-------+--------------------+-------+-------+
|ezsdavt|If anyone is look...|2019-09|tickets|
|ezsdavt|If anyone is look...|2019-09|   👀👀|
|ezxy2ms|            Jack off|2019-09|   jack|
|f00o5zr|    Where’s the joke|2019-09| wheres|
|f00o5zr|    Where’s the joke|2019-09|   joke|
+-------+--------------------+-------+-------+
only showing top 5 rows



In [9]:
# **TODO** STEP 7: Count the number of  times a token appears in each comment.
# How would you do it in SQL? If you're stuck, write out the SQL query and find the proper
# algebraic operators in Spark. *Alias* the resulting column "tf_num"
sel = ['id','word','month','body']

token_count_in_comments = result5.groupBy(sel).count().select(*sel,F.col('count').alias('tf_num'))
token_count_in_comments.show(5)

+-------+-------+-------+--------------------+------+
|     id|   word|  month|                body|tf_num|
+-------+-------+-------+--------------------+------+
|f026ytg| couple|2019-09|that’s the minor ...|     1|
|ez6zh9g|signing|2019-09|If I’m not mistak...|     1|
|ez6zh9g| spring|2019-09|If I’m not mistak...|     1|
|etih7xu|    mac|2019-07|Wow... my dumb as...|     1|
|err90z5|      5|2019-06|4 or 5 for AB is ...|     2|
+-------+-------+-------+--------------------+------+
only showing top 5 rows



In [10]:
# **TODO** STEP 8: Count the unique number of tokens in each comment. *Alias* the resulting
# column "tf_den"
sel = ['id','month','body']
tokens_in_comment = token_count_in_comments.groupBy(sel).count().select(*sel, F.col('count').alias('tf_den'))
tokens_in_comment.show(5)

+-------+-------+--------------------+------+
|     id|  month|                body|tf_den|
+-------+-------+--------------------+------+
|eig34t6|2019-03|I get where you’r...|    27|
|ezk2maf|2019-09|Even better advic...|    13|
|egzmds8|2019-02|&gt; The time and...|    56|
|eeav86c|2019-01|That is one way t...|    23|
|f7dm3hy|2019-11|I'm infuriated th...|    27|
+-------+-------+--------------------+------+
only showing top 5 rows



In [11]:
# **TODO** STEP 9: Count the number of comments in each month-year and alias the resulting
# column as "idf_num".
comments_in_month = tokens_in_comment.groupBy('month').count().select('month',F.col('count').alias('idf_num'))
comments_in_month.show(5)

+-------+-------+
|  month|idf_num|
+-------+-------+
|2019-10|   5108|
|2019-11|   4528|
|2019-03|   6116|
|2019-07|   4580|
|2019-05|   5006|
+-------+-------+
only showing top 5 rows



In [12]:
# **TODO** STEP 10: For each month-year, calculate the number of comments each unique token appears in.
# This is the denominator of the IDF term, so alias the resulting column "idf_den". IDF can be very
# noisy, so filter so only tokens that have an idf_den > 7 are included. This also speeds up execution.
sel = ['month','word']
comments_with_token = token_count_in_comments.groupBy(sel).count().where("count > 7").select(*sel, F.col('count').alias('idf_den'))
comments_with_token.show(5)

+-------+--------+-------+
|  month|    word|idf_den|
+-------+--------+-------+
|2019-03|   moved|      8|
|2019-08|   youll|     89|
|2019-02|textbook|     10|
|2019-03|    type|     36|
|2019-02|   check|     70|
+-------+--------+-------+
only showing top 5 rows



In [13]:
# **TODO** STEP 11: You now have 4 different tables containing the components of TF-IDF. You need
# to join them together. This data is small, but we still need to be very careful.
# Before believing your join is correct, add .explain() at the end of your statement to make sure
# Spark does the right thing. If it doesn't, you will may need to change a default (see Lecture 15)
# and also provide a join hint to the optimizer. Hint: you probably don't need to worry about this,
# if you do, use *broadcast*. The data is small enough that this shouldn't be an issue.
# NOTE: This should take about 5-10 minutes on a modern system once you call .collect(), .head(),
# or .show(). Remember, due to LAZY EVALUATION, the join command will return automatically.
id_and_tf_den = tokens_in_comment.select('id','tf_den')
word_month_idf_den = comments_with_token.select('word','month','idf_den')

sel = ['L.id', 'word', 'month', 'body', 'tf_num', 'tf_den']
num_den = token_count_in_comments.alias('L').join(broadcast(id_and_tf_den.alias('R')), col('L.id') == col('R.id')).select(sel)

sel = ['L.id', 'word', 'L.month', 'L.body', 'tf_num', 'tf_den','idf_num']
num_den_idf = num_den.alias('L').join(broadcast(comments_in_month.alias('R')), col('L.month') == col('R.month')).select(sel)


sel = ['L.id', 'L.word', 'L.month', 'L.body', 'L.tf_num', 'L.tf_den','L.idf_num', 'idf_den']
joined = num_den_idf.alias('L').join(broadcast(word_month_idf_den.alias('R')), (col('L.month') == col('R.month')) & (col('L.word') == col('R.word'))).select(sel)
joined.show(5)

+-------+--------+-------+--------------------+------+------+-------+-------+
|     id|    word|  month|                body|tf_num|tf_den|idf_num|idf_den|
+-------+--------+-------+--------------------+------+------+-------+-------+
|f026ytg|  couple|2019-09|that’s the minor ...|     1|     8|   6103|     65|
|ez6zh9g|  spring|2019-09|If I’m not mistak...|     1|    17|   6103|     49|
|err90z5|       5|2019-06|4 or 5 for AB is ...|     2|     9|   4781|     58|
|f9teu3v|mistakes|2019-12|I guarantee almos...|     1|    35|   3718|      9|
|ex0d0uv|personal|2019-08|I can't exactly s...|     1|    98|   4874|     34|
+-------+--------+-------+--------------------+------+------+-------+-------+
only showing top 5 rows



In [17]:
# **TODO** STEP 12: Compute TF-IDF. See the formula in the spec. The log function is imported for you.
# There should be one TF-IDF value for each month-year/comment/token triple. Call the new column "tfidf".
tfidf_df = joined.withColumn('tfidf', (joined.tf_num/joined.tf_den)*log(joined.idf_num/(1+joined.idf_den)))
tfidf_df.show(5)

+-------+--------+-------+--------------------+------+------+-------+-------+--------------------+
|     id|    word|  month|                body|tf_num|tf_den|idf_num|idf_den|               tfidf|
+-------+--------+-------+--------------------+------+------+-------+-------+--------------------+
|f026ytg|  couple|2019-09|that’s the minor ...|     1|     8|   6103|     65|  0.5658601238147587|
|ez6zh9g|  spring|2019-09|If I’m not mistak...|     1|    17|   6103|     49|  0.2826183957127264|
|err90z5|       5|2019-06|4 or 5 for AB is ...|     2|     9|   4781|     58|  0.9766372366045297|
|f9teu3v|mistakes|2019-12|I guarantee almos...|     1|    35|   3718|      9| 0.16909588786535268|
|ex0d0uv|personal|2019-08|I can't exactly s...|     1|    98|   4874|     34|0.050370634415262634|
+-------+--------+-------+--------------------+------+------+-------+-------+--------------------+
only showing top 5 rows



In [18]:
# **TODO** STEP 13: Having a TF-IDF score for each token in each comment in each time period is correct;
# however, it doesn't get us what we want. Let's now compute the average TF-IDF value of each token per month-year.
# *Alias* the column "avg_tfidf".
sel = ['word','month',F.col("avg(tfidf)").alias('avg_tfidf')]
avg_df = tfidf_df.groupBy('word', 'month').agg(F.avg('tfidf')).select(sel)
avg_df.show(5)

+--------+-------+-------------------+
|    word|  month|          avg_tfidf|
+--------+-------+-------------------+
|accepted|2019-08|0.29177452105790985|
|    push|2019-10| 0.7640444673835423|
|yourself|2019-11|0.20341916475253005|
| regents|2019-03| 0.4637794895577473|
|    lazy|2019-03|0.24533689903449543|
+--------+-------+-------------------+
only showing top 5 rows



In [19]:
# **TODO** STEP 13: Find the token(s) that have maximal average-TF-IDF within each month-year.
# This is a mathematical argmax, and can be implemented as a join.
# You will need to use a join on table avg_tfidf. Again, see the cautions in Step 11.
max_tfidf = avg_df.groupBy('month').max('avg_tfidf').select('month',F.col('max(avg_tfidf)').alias('max_avg_tfidf'))
# max_tfidf.show(5)
max_joined = max_tfidf.alias('L').join(broadcast(avg_df).alias('R'),col('L.max_avg_tfidf') == col('R.avg_tfidf')).select('L.month','R.word','max_avg_tfidf')
max_joined.show(12)

+-------+---------+------------------+
|  month|     word|     max_avg_tfidf|
+-------+---------+------------------+
|2019-10|       😂| 3.285918177327779|
|2019-11|       🦃| 5.584822275826805|
|2019-03|     amen|  5.53067225746926|
|2019-07|copypasta|2.8365135111639743|
|2019-05|      lt3|2.8182651663901304|
|2019-08|     bruh| 2.487095325256426|
|2019-01|      yep|1.9305518351414224|
|2019-09|       ty| 3.732987144711901|
|2019-06| messaged|3.5813949885185186|
|2019-04|      wtf|3.2384386326871994|
|2019-12|   please|3.0570872091281602|
|2019-02|      wtf|2.6818136326206155|
+-------+---------+------------------+



In [20]:
# Note that Spark may give you some problems in Step 13. If you get errors that you are referring to a non-existent
# column, particularly in the join to create max_joined, what's happening is that Spark sees multiple copies
# of the same column and isn't sure which column you're referring to. Use the following snippet (or something
# similar to it) if that happens, before moving on to the final join.
# max_tfidf = max_tfidf.select(F.col("month").alias("month"), "max")
# Depending on how you wrote your code, you may not run into this issue.

# **TODO** STEP 14: Finally, order the results by month and output just the month-year and token.
# Note that some months may be missing due to the earlier filter on IDF.
final_result = max_joined.orderBy(col('month').asc()).select('month','word')
final_result.show()

### EXECUTING MY SOLUTION TOOK APPROXIMATELY 10-15 MINUTES FROM TOP TO BOTTOM.
# You should make sure a step works correctly before moving to the next.
# This will save you time.

+-------+---------+
|  month|     word|
+-------+---------+
|2019-01|      yep|
|2019-02|      wtf|
|2019-03|     amen|
|2019-04|      wtf|
|2019-05|      lt3|
|2019-06| messaged|
|2019-07|copypasta|
|2019-08|     bruh|
|2019-09|       ty|
|2019-10|       😂|
|2019-11|       🦃|
|2019-12|   please|
+-------+---------+

